In [1]:
# libraries

from io import StringIO
from tqdm.notebook import tqdm_notebook as tqdm

import re
import nltk
import numpy as np
import pandas as pd
import requests
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
## checking if pytorch can use the gpu

# print(torch.cuda.current_device())
# print(torch.cuda.device(0))
# print(torch.cuda.get_device_name(0))
# print(torch.cuda.is_available())

# Constructing classes

In this section, we build classes which are later used to load and clean data. We moved this to the top of the notebook to declutter the following section and to make it easier for you to follow our workflow.

In [3]:
# class to load data
# following this stackoverflow post to download directly from Google Drive:
# https://stackoverflow.com/a/56611995

class DataLoader:
    def __init__(self, gdrive_url):
        """
        Class constructor
        Args:
            gdrive_url (str): URL to share Google Drive docs
        """
        self.path = gdrive_url   
    def load_csv(self):
        """
        Returns DataFrame containing data from csv
        Args:
            None
        """
        file_id = self.path.split('/')[-2]
        dwn_url='https://drive.google.com/uc?export=download&id=' + file_id
        url = requests.get(dwn_url).content
        csv_raw = StringIO(url.decode('utf-8'))
        df_ta = pd.read_csv(csv_raw)
        return(df_ta)
    
    def load_txt(self):
        """
        Returns String containing data from txt
        Args:
            None
        """
        file_id = self.path.split('/')[-2]
        dwn_url='https://drive.google.com/uc?export=download&id=' + file_id
        #print(dwn_url)
        url = requests.get(dwn_url).content
        scifi = StringIO(url.decode('utf-8')).getvalue()
        return(scifi)

#txt = DataLoader("https://drive.google.com/file/d/10ehW4jZND3QA29v9aNboYUett5-swuNe/view?usp=sharing").load_txt()

In [4]:
# class to clean data
# when cleaning the text data, we proceed as suggested in McMahan (Natural Language Processing with PyTorch)
# by converting each sentence to lowercase and removing the punctuation completely. other non-alpha characters are also removed 
# white space between words is used to split the strings based on it to retrieve a list of tokens

# stopword removal inspired by this stackoverlow answer
# https://stackoverflow.com/a/43407993/7505264
from nltk.corpus import stopwords
stop = stopwords.words('english')
tqdm.pandas()

class DataCleaner:
    # this method removes non alpha characters like '#', ';', ',','.','/' etc.
    def remove_nonalpha_chars(self, df, column):
        """
        Returns a processed DataFrame, does not change original
        Removes non-alpha characters
        Args:
            df (DataFrame): DataFrame containing the corpus
            column (str): Name of the DF column with text data (corpus)
        """
        
        df_out = df.copy(deep=True)
        df_out[column] = df_out[column].progress_apply(lambda x: re.sub(r"\s*[^A-Za-z]+\s*", " ", x))
        return df_out
    
    def lower_casing(self, df, column):
        """
        Returns a processed DataFrame, does not change original
        Turns strings to lower case
        Args:
            df (DataFrame): DataFrame containing the corpus
            column (str): Name of the DF column with text data (corpus)
        """
        
        df_out = df.copy(deep=True)
        # we turn all words to lower case to avoid that words that are at the beginning of a sentence are seen as different words
        # since we are dealing with English language which is not capitalizing nouns 
        # However, proper nouns are capitalized in English languange
        df_out[column] = df_out[column].progress_apply(lambda x: x.lower())
        return df_out
    
    def remove_stopwords(self, df, column):
        """
        Returns a processed DataFrame, does not change original
        Removes stopwords from text in English language
        Args:
            df (DataFrame): DataFrame containing the corpus
            column (str): Name of the DF column with text data (corpus)
        """
        df_out = df.copy(deep=True)
        df_out[column] = df_out[column].progress_apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
        return df_out
    
        

/home/tore/venv/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [5]:
## Testing class
# df_ta['Review'].apply(lambda x: re.sub(r"\s*[^A-Za-z]+\s*", " ",x))

# rvws = {'Review': ['TEST_.']}

# df_test = pd.DataFrame(rvws, columns = ['Review'])

# df_test['Review'].apply(lambda x: x.lower())

In [6]:
# Vectorizer class
class CBOWVectorizer:
    """ Class that coordinates Vocabularies by vectorizing """
    
    def vectorize(self, df, column, window_size):
        """
        Args:
            df (DataFrame): dataframe containing the text samples
            column (str): name of the column that contains text samples
            window_size (int): length of the word window 
                (2 means 2 leading and 2 trailing words, hence, 4 context words)
        """
    
    
    


# Loading data


In [7]:
# url for tripadvisor_hotel_reviews.csv
# following this stackoverflow post to download directly from Google Drive:
# https://stackoverflow.com/a/56611995

### URLS
orig_url_ta = 'https://drive.google.com/file/d/1ihP1HZ8YHVGGIEp1RHxXdt3PPIi12xvL/view?usp=sharing'
orig_url_scifi = "https://drive.google.com/file/d/10ehW4jZND3QA29v9aNboYUett5-swuNe/view?usp=sharing"

### DataLoaders
TravAdvDataSetLoader = DataLoader(orig_url_ta)
ScifiLoader = DataLoader(orig_url_scifi)

### CSV and txts
df_ta = TravAdvDataSetLoader.load_csv()
scifi_txt = ScifiLoader.load_txt()

# file_id = orig_url_ta.split('/')[-2]
# dwn_url='https://drive.google.com/uc?export=download&id=' + file_id
# url = requests.get(dwn_url).content
# csv_raw = StringIO(url.decode('utf-8'))
# df_ta = pd.read_csv(csv_raw)

# Exploratory data analysis

Just checking if the download of the csv was successful by looking at the first and last lines.

In [9]:
df_ta.head()

,Review,Rating
0,nice hotel expensive parking got good deal sta...,4
1,ok nothing special charge diamond member hilto...,2
2,nice rooms not 4* experience hotel monaco seat...,3
3,"unique, great stay, wonderful time hotel monac...",5
4,"great stay great stay, went seahawk game aweso...",5


**Observations**:


1.   Some reveiw include the rating (i.e. 4*). This should be removed
2.   The last line has a typo (and probably many other lines too) which add noise. A correction of all errors, however, is not realistic.


Now, we look for all characters used in the reviews to get an idea of how we need to preprocess the data. We can see that there are no foreign language characters in the data but a couple of symbols, special characters and emojis.

(this takes a minute to run, so skip this cell if you are in a hurry)

In [10]:
# the function concatenates all strings and then builds a set of unique characters
# np.array(df_ta.Review.sum()).reshape(-1, 5)

**Observations**:

We see that there are certain non-alpha signs, for example € or $, which could add value to the word embedding as they actually stand for Euro or Dollar and make a lot of sense in the given context.

# Data Preprocessing

## Data Cleaning

For the data preprocessing, we first create a class that helps us to clean the data (following the OOP approach).

**Note**: We only perform operations on the complete data set (training + test set) which do not lead to information leakage. Removing certain characters from the test set is a valid operation that also occurs in real world setting. Data is usually preprocessed before predictions are made.

After cleaning the data in the step below, we compare the results of the second review to confirm that the cleaning was successful.
We realize that most comments contain typos and that some typos like "did n't" result in single characters in the corpus. Given that we cannot correct every typo, we accept this noise in our data. 

In [11]:
DtCleaner = DataCleaner()

df_ta_cl = DtCleaner.remove_nonalpha_chars(df_ta, 'Review')
df_ta_cl = DtCleaner.lower_casing(df_ta_cl, 'Review')
df_ta_cl = DtCleaner.remove_stopwords(df_ta_cl, 'Review')

df_ta_cl.iloc[1]['Review']

'ok nothing special charge diamond member hilton decided chain shot th anniversary seattle start booked suite paid extra website description suite bedroom bathroom standard hotel room took printed reservation desk showed said things like tv couch ect desk clerk told oh mixed suites description kimpton website sorry free breakfast got kidding embassy suits sitting room bathroom bedroom unlike kimpton calls suite day stay offer correct false advertising send kimpton preferred guest website email asking failure provide suite advertised website reservation description furnished hard copy reservation printout website desk manager duty reply solution send email trip guest survey follow email mail guess tell concerned guest staff ranged indifferent helpful asked desk good breakfast spots neighborhood hood told hotels gee best breakfast spots seattle block away convenient hotel know exist arrived late night pm inside run bellman busy chating cell phone help bags prior arrival emailed hotel inf

## Training/Test Data Set Preparation

In [13]:
# splitting the data into a training and test set
# using set seed to allow replication
np.random.seed(123)
m = np.random.rand(len(df_ta_cl)) < 0.7

df_ta_train = df_ta_cl[m]
df_ta_test = df_ta_cl[~m]

In [14]:
# just checking if split was correct
len(df_ta_train) / len(df_ta)

0.7005026597042604

In [15]:
corpus_ta_train = df_ta_train['Review'].str.cat(sep=', ')
corpus_ta_test =  df_ta_test['Review'].str.cat(sep=', ')

corpus_ta = corpus_ta_train + corpus_ta_test

In [16]:
# we are checking the ration of training to test sample again because we split the dataframe above, not the corpus
# in theory, we could have sampled a lot of rows from the DF with long strings and obtained a training set which is more than
# 70% of the corpus. highly unlikely given the size of the DF and the random sampling. So just to make sure we got this right.

len(corpus_ta_train) / (len(corpus_ta_train) + len(corpus_ta_test))

0.7001140124630911

In [17]:
print("n voc train: " + str(len(set(corpus_ta_train.split()))))
print("n voc test: " + str(len(set(corpus_ta_test.split()))))

print("n voc combined: " + str(len(set((corpus_ta_train + corpus_ta_test).split()))))

len(set(corpus_ta_test.split()).difference(set(corpus_ta_train.split())))

n voc train: 44035
n voc test: 28346
n voc combined: 52512


8476

In [19]:
# Vocabulary size and number of words in a sequence.
vocab_size = 10000
sequence_length = 100

# Use the text vectorization layer to normalize, split, and map strings to 
# integers. Note that the layer uses the custom standardization defined above. 
# Set maximum_sequence length as all samples are not of the same length.
"""vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)"""

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
#text_ds = train_ds.map(lambda x, y: x)
#vectorize_layer.adapt(text_ds)

"vectorize_layer = TextVectorization(\n    standardize=custom_standardization,\n    max_tokens=vocab_size,\n    output_mode='int',\n    output_sequence_length=sequence_length)"

Encoding our vocabulary. We are encoding the full corpus, as suggested in the exercise forum.

In [20]:
vocab_ta = set(corpus_ta.split())
vocab_ta_size = len(vocab_ta)

word_to_ix = {word: i for i, word in enumerate(vocab_ta)}

In [21]:
# Turning the corpus into training and test data 
CONTEXT_SIZE = 2

In [22]:
class VectorizerCBOW:
    
    def vectorize(self, context_size, corpus):
        """
        Returns:
            list of context words and the corresponding central words
        Args:
            contet_size (int): the window length to the left and to the right of the central word
            corpus (str): the cleaned corpus as on string, words seperated by space
        """
        
        # first, extract the context words and the corresponding central words
        data = []
        corpus_splt = corpus.split()
        for i in range(2, len(corpus_splt) - 2):
            context = [corpus_splt[i - 2], corpus_splt[i - 1],
                       corpus_splt[i + 1], corpus_splt[i + 2]]
            target = corpus_splt[i]
            data.append((context, target))
        
        return df

In [26]:
class VectorizerCBOW:
    
    def vectorize(self, context_size, corpus):
        """
        Returns:
            list of context words and the corresponding central words
        Args:
            contet_size (int): the window length to the left and to the right of the central word
            corpus (str): the cleaned corpus as on string, words seperated by space
        """
        
        # first, extract the context words and the corresponding central words
        data = []
        corpus_splt = corpus.split()
        for i in range(2, len(corpus_splt) - 2):
            context = [corpus_splt[i - 2], corpus_splt[i - 1],
                       corpus_splt[i + 1], corpus_splt[i + 2]]
            target = corpus_splt[i]
            data.append((context, target))
        
        return data
        
        
VectCBOW = VectorizerCBOW()

cont_targ_train = VectCBOW.vectorize(CONTEXT_SIZE, corpus_ta_train)


Until here, everything works
Below, work in progress

In [27]:

class CBOW(nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_size):
        super(CBOW, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))  
        # -1 implies size inferred for that index from the size of the data
        #print(np.mean(np.mean(self.linear2.weight.data.numpy())))
        out1 = F.relu(self.linear1(embeds)) # output of first layer
        out2 = self.linear2(out1)           # output of second layer
        #print(embeds)
        log_probs = F.log_softmax(out2, dim=1)
        return log_probs
    
    def predict(self, input):
        context_idxs = torch.tensor([word_to_ix[w] for w in input], dtype=torch.long)
        res = self.forward(context_idxs)
        res_arg = torch.argmax(res)
        res_val, res_ind = res.sort(descending=True)
        res_val = res_val[0][:3]
        res_ind = res_ind[0][:3]
        #print(res_val)
        #print(res_ind)
        for arg in zip(res_val,res_ind):
            #print(arg)
            print([(key,val,arg[0]) for key,val in word_to_ix.items() if val == arg[1]])


In [25]:
EMBED_DIM = 50

losses = []
loss_function = nn.NLLLoss()
model = CBOW(vocab_ta_size, EMBED_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

# Freeze embedding layer
#model.freeze_layer('embeddings')

for epoch in range(1):
    total_loss = 0
    #------- Embedding layers are trained as well here ----#
    #lookup_tensor = torch.tensor([word_to_ix["poor"]], dtype=torch.long)
    #hello_embed = model.embeddings(lookup_tensor)
    #print(hello_embed)
    # -----------------------------------------------------#
    for context, target in cont_targ_train:

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)
        #print("Context id",context_idxs)

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_idxs)
        #print(log_probs)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))
        #print(loss)

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    print(total_loss)
    losses.append(total_loss)
#print(losses)  # The loss decreased every iteration over the training data!

#Print the model layer parameters
#model.print_layer_parameters()

#Predict the next word given n context words
model.predict(['of','all','human'])


RuntimeError: size mismatch, m1: [1 x 200], m2: [100 x 128] at /pytorch/aten/src/TH/generic/THTensorMath.cpp:41